In [1]:
import math 
import numpy as np
from numpy import random

from ipywidgets import *
import ipywidgets as widgets
from ipysheet import from_array, to_array, sheet, cell

In [2]:
random.seed()

In [3]:
#поступление запросов 
A = widgets.FloatSlider(min=0.0, max=10.0, step=0.5, value = 1.0)
B = widgets.FloatSlider(min=0.0, max=10.0, step=0.5, value = 3.0)

display(A, B)
a = A.value 
b = B.value 

if (b < a):
    b, a = a, b


FloatSlider(value=1.0, max=10.0, step=0.5)

FloatSlider(value=3.0, max=10.0, step=0.5)

In [4]:
#сколько моделируем систему 
T = widgets.FloatSlider(min=0.0, max=10000.0, value = 500.0)
display(T)

FloatSlider(value=500.0, max=10000.0)

In [5]:
#обработка сpp
CA = widgets.FloatSlider(min=0.0, max=10.0, step=0.5, value = 1.0)
CB = widgets.FloatSlider(min=0.0, max=10.0, step=0.5, value = 3.0)
display(CA, CB)

FloatSlider(value=1.0, max=10.0, step=0.5)

FloatSlider(value=3.0, max=10.0, step=0.5)

In [6]:
#обработка web
WA = widgets.FloatSlider(min=0.0, max=10.0, step=0.5, value = 4.0)
WB = widgets.FloatSlider(min=0.0, max=10.0, step=0.5, value = 5.0)
display(WA, WB)

FloatSlider(value=4.0, max=10.0, step=0.5)

FloatSlider(value=5.0, max=10.0, step=0.5)

In [7]:
#время работы докера
DA = widgets.FloatSlider(min=0.0, max=10.0, step=0.5, value = 0.0)
DB = widgets.FloatSlider(min=0.0, max=10.0, step=0.5, value = 10.0)

display(DA, DB)

FloatSlider(value=0.0, max=10.0, step=0.5)

FloatSlider(value=10.0, max=10.0, step=0.5)

In [8]:
def generate_next(a, b):
    return random.uniform(a, b)

In [9]:
mt = 0 #время моделирования 
#соотношение заявок
ra = 5
rb = 3

#количество заявок
reqa = 0 #заявки первого типа
reqb = 0 #заявки второго типа

#очереди
que_a = 0
que_b = 0

#флаги событий
ba = False #нет заявок первого типа
bb = False #нет заявок второго типа
bd = False #докер не занят

#время происхождения событий
t_req = generate_next(a, b) #когда следующая заявка
t_ra = math.inf
t_rb = math.inf
t_c = math.inf

class typeReq:
    def __init__(self, name, time):
        self.name = name
        self.time = time

#типы событий
ev_req = typeReq("req", t_req) #приходит заявка

ev_ra = typeReq("cpp", t_ra) #она либо С++
ev_rb = typeReq("web", t_rb) #либо веб

ev_c = typeReq("docker", t_c) #докер старается взять сначала с++

times = [ev_req, ev_ra, ev_rb, ev_c]

In [10]:
while mt <= T.value:
    #обновили время
    mt += times[0].time
    
    for i in range(len(times) - 1, -1, -1):
        times[i].time -= times[0].time
    
    if times[0].name == 'req':
        ev_req.time = generate_next(a, b) #ждем следующую заявку
        P = random.random()
        #пришла заявка первого типа
        if (P <= (ra/ (ra+rb))):
            ba = True #есть заявка первого типа
            reqa += 1
            ev_ra.time = generate_next(CA.value, CB.value)
            
        else: #пришла заявка второго типа
            bb = True #есть заявка второго типа
            reqb += 1
            ev_rb.time = generate_next(WA.value, WB.value)
    
    elif times[0].name == 'cpp':
        ba = False
        ev_ra.time = math.inf
        #можем сразу обработать 
        if not bd:
            bd = True
            ev_c.time = generate_next(DA.value, DB.value)
        else:
            que_a += 1
            
    elif times[0].name == 'web':
        bb = False
        ev_rb.time = math.inf
        #можем сразу обработать 
        if not bd:
            bd = True
            ev_c.time = generate_next(DA.value, DB.value)
        else:
            que_b += 1
    elif times[0].name == 'docker':
        bd = False #докер освободился
        ev_c.time = math.inf
        #если есть заявки первого типа: обрабатываем в первую очередь
        if que_a > 0:
            bd = True #заняли докер
            que_a -= 1
            ev_c.time = generate_next(DA.value, DB.value) #посмотрели, когда он освободится
        elif que_b > 0: #только если нет легковесных заявок
            bd = True #заняли докер
            que_b -= 1
            ev_c.time = generate_next(DA.value, DB.value) #посмотрели, когда он освободится
   
    times = sorted(times, key = (lambda x: x.time) )

In [11]:
s = sheet(5, 1, column_headers = list(['Сpp', 'Web', 'Время моделирования', 'Осталось заявок CPP', 'Осталось заявок WEB']))
cell(0, 0, reqa)
cell(0, 1, reqb) 
cell(0, 2, T.value)
cell(0, 3, que_a)
cell(0, 4, que_b)
s

Sheet(cells=(Cell(column_end=0, column_start=0, row_end=0, row_start=0, type='numeric', value=140), Cell(colum…